The reward function class generates an instantaneous reward given a 
    particular state based on the following parameters
    
    1. Legality of state:
        - x for x overlaps
    
    2. Conservation of water:
        - for each water used
        
    3. Diversity of garden:
        + x for x > 3 plant species
        0 for 3 species
        - x for x < 3 plant species
    
    4. Density of garden
        +% for %density > 75
        -% for %density < 75

fuhewufewhfewiuhf

In [ ]:
class reward():
    def __init__(self, discount):
        self.discount = discount
        
    def legality(self, state):
        return 0
    
    def water(self, action):
        return 0
    
    def diversity(self, state):
        return 0
    
    def density(self, state):
        return 0
    
    def get_reward(self, state, action):
        p1 = self.legality(state)
        p2 = self.water(action)
        p3 = self.diversity(state)
        p4 = self.density(state)
        
        return p1 + p2 + p3 + p4